In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext line_profiler
import line_profiler

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import HOTS.KmeansCluster as KmeansCluster
from HOTS.Event import Event, SimpleAlphabet
from HOTS.STS import STS
from HOTS.Layer import ClusteringLayer, FilterNHBD
from HOTS.Monitor import (
    DisplayImage,
    DisplaySurface3D,
    DisplaySurface2D,
    DisplayConvergence,
)
from HOTS.Tools import SaveObject, LoadObject
from HOTS.Classifier import Classifier
from HOTS.Network import Network
from HOTS.KmeansCluster import KmeansMaro

In [3]:
tau = 0.02
R = 2
nb_cluster = [4, 8, 16]
ImageSize = (32, 32)

# 1.Profiling the event loading

In [4]:
event_tr, event_te, event_cl, label_tr, label_te = SimpleAlphabet(
    NbTrainingData=36,
    NbTestingData=40,
    ClusteringData=[1, 32, 19, 22, 29],
    OutOnePolarity=False,
    verbose=0,
    Path="../Data/ExtractedStabilized.mat",
    LabelPath="../Data/alphabet_label.pkl",
)

In [5]:

%lprun -T ProfilingFiles/timing_load_event.txt -f event_te.LoadFromMat -f SimpleAlphabet event_tr, event_te, event_cl, label_tr, label_te = \
                                                        SimpleAlphabet(NbTrainingData=36, NbTestingData=40,\
                                                        ClusteringData=[1, 32, 19, 22, 29],\
                                                        OutOnePolarity = False, verbose=0, \
                                                        Path='../Data/ExtractedStabilized.mat',\
                                                        LabelPath='../Data/alphabet_label.pkl')


*** Profile printout saved to text file 'ProfilingFiles/timing_load_event.txt'. 


Timer unit: 1e-06 s

Total time: 0.356263 s
File: /Users/laurentperrinet/quantic/science/SpikeAI/HOTS/HOTS/Event.py
Function: LoadFromMat at line 38

Line #      Hits         Time  Per Hit   % Time  Line Contents
    38                                               def LoadFromMat(self, path, image_number, verbose=0):
    39                                                   '''
    40                                                   Load Events from a .mat file. Only the events contained in ListPolarities are kept:
    41                                                   INPUT
    42                                                       + path : a string which is the path of the .mat file (ex : './data_cache/ExtractedStabilized.mat')
    43                                                       + image_number : list with all the numbers of image to load
    44                                                   '''
    45         3     195507.0  65169.0     54.9          obj = scipy.io.l

In [6]:
%cat ProfilingFiles/timing_load_event.txt

Timer unit: 1e-06 s

Total time: 0.356263 s
File: /Users/laurentperrinet/quantic/science/SpikeAI/HOTS/HOTS/Event.py
Function: LoadFromMat at line 38

Line #      Hits         Time  Per Hit   % Time  Line Contents
    38                                               def LoadFromMat(self, path, image_number, verbose=0):
    39                                                   '''
    40                                                   Load Events from a .mat file. Only the events contained in ListPolarities are kept:
    41                                                   INPUT
    42                                                       + path : a string which is the path of the .mat file (ex : './data_cache/ExtractedStabilized.mat')
    43                                                       + image_number : list with all the numbers of image to load
    44                                                   '''
    45         3     195507.0  65169.0     54.9          obj = scipy.io.l

# 2.Profiling the Network Training (with KmeansLagorce Algo)

In [7]:
InputFilter = FilterNHBD(threshold=15, neighbourhood=2)
L1 = ClusteringLayer(
    tau=tau,
    R=R,
    verbose=1,
    ThrFilter=0,
    LearningAlgo="lagorce",
    kernel="exponential",
    eta=None,
    eta_homeo=None,
    sigma=None,
)
L2 = ClusteringLayer(
    tau=10 * tau,
    R=2 * R,
    verbose=1,
    ThrFilter=0,
    LearningAlgo="lagorce",
    kernel="exponential",
    eta=None,
    eta_homeo=None,
    sigma=None,
)
L3 = ClusteringLayer(
    tau=10 * 10 * tau,
    R=2 * 2 * R,
    verbose=1,
    ThrFilter=0,
    LearningAlgo="lagorce",
    kernel="exponential",
    eta=None,
    eta_homeo=None,
    sigma=None,
)
Net = Network([InputFilter, L1, L2, L3])

In [8]:
"""
%lprun -T ProfilingFiles/timing_NetworkTraining_v0.txt -f Net.TrainCluster -f L1.TrainLayer -f L1.ClusterLayer.fit\
                                -f L1.ClusterLayer.predict \
                                ClusterLayer,event_output = \
                                Net.TrainCluster(event = event_cl, NbClusterList = nb_cluster,\
                                record_each=100, NbCycle=1)
"""

'\n%lprun -T ProfilingFiles/timing_NetworkTraining_v0.txt -f Net.TrainCluster -f L1.TrainLayer -f L1.ClusterLayer.fit                                -f L1.ClusterLayer.predict                                 ClusterLayer,event_output =                                 Net.TrainCluster(event = event_cl, NbClusterList = nb_cluster,                                record_each=100, NbCycle=1)\n'

In [9]:
%cat ProfilingFiles/timing_NetworkTraining_v1.txt

Timer unit: 1e-06 s

Total time: 32.3982 s
File: /Users/boutin.v/Documents/HOTS/HOTS/KmeansCluster.py
Function: fit at line 85

Line #      Hits         Time  Per Hit   % Time  Line Contents
    85                                               def fit (self,STS, init=None, NbCycle=1):
    86                                                   '''
    87                                                   Methods to learn prototypes fitting data
    88                                                   INPUT :
    89                                                       + STS : (<STS object>) Stream of SpatioTemporal Surface to fit
    90                                                       + init : (<string>) Method to initialize the prototype ('rdn' or None)
    91                                                       + NbCycle : (<int>) Number of time the stream is going to be browse.
    92                                                   OUTPUT :
    93                                

In [10]:
%cat ProfilingFiles/timing_NetworkTraining_v0.txt

Timer unit: 1e-06 s

Total time: 155.346 s
File: /Users/boutin.v/Documents/HOTS/HOTS/KmeansCluster.py
Function: fit at line 39

Line #      Hits         Time  Per Hit   % Time  Line Contents
    39                                               def fit (self,STS, init=None, NbCycle=1):
    40                                                   '''
    41                                                   Methods to learn prototypes fitting data
    42                                                   INPUT :
    43                                                       + STS : (<STS object>) Stream of SpatioTemporal Surface to fit
    44                                                       + init : (<string>) Method to initialize the prototype ('rdn' or None)
    45                                                       + NbCycle : (<int>) Number of time the stream is going to be browse.
    46                                                   OUTPUT :
    47                                

# 3.Profiling the Network Running

In [11]:
#%lprun -T ProfilingFiles/timing_NetworkRunning.txt -f Net.RunNetwork -f L1.RunLayer\
#                event_o_tr = Net.RunNetwork(event_tr, ClusterList=ClusterLayer)

In [12]:
%lprun -T ProfilingFiles/timing_NetworkRunning_v1.txt -f Net.RunNetwork -f L1.RunLayer\
                event_o_tr = Net.RunNetwork(event_tr, ClusterList=ClusterLayer)

NameError: name 'ClusterLayer' is not defined

In [ ]:
%cat ProfilingFiles/timing_NetworkRunning_v1.txt

In [ ]:
# %load ProfilingFiles/timing_NetworkRunning.txt

# 4.Profiling the Classification

In [ ]:
event_o_te = Net.RunNetwork(event_te, ClusterList=ClusterLayer)
Classif0 = Classifier(
    event_o_tr, event_o_te, TrainingLabel=label_tr, GroundTruth=label_te
)

In [ ]:

%lprun -T ProfilingFiles/timing_NetworkClassifier.txt -f Classif0.HistogramDistance \
                prediction, accuracy, method = Classif0.HistogramDistance(to_print=True)

In [ ]:
%cat ProfilingFiles/timing_NetworkClassifier.txt